In [2]:
from dotenv import load_dotenv
env = load_dotenv()

In [38]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai import ChatOpenAI
from IPython.display import Image
import IPython

In [37]:
# Tools 

In [41]:
from typing import TypedDict, Annotated, Dict, Any
import operator

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    # output: str

In [56]:
# Download PDF tool
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.pydantic_v1 import BaseModel, Field
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

search_tool = TavilySearchResults()

class ScrapeInput(BaseModel):
    url: str = Field(description="the URL of the menu page")

@tool("scrape-pdf", args_schema=ScrapeInput, return_direct=True)
def scrape_pdf(url: str):
    """Scrape a webpage that may include links to a restaurants current menu and return the links"""
    pdf_links = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        pdf_links = [link.get('href') for link in links if link.get('href').endswith('.pdf')]
    except:
        print(f"failed to scrape {filename}")
    return pdf_links

class DowloadInput(BaseModel):
    url: str = Field(description="the URL of the menu")
    filename: str = Field(description="the name of the restaurants")

@tool("download-pdf", args_schema=DowloadInput, return_direct=True)
def download_pdf(url: str, filename: str):
    """Download a pdf file from a given url and filename"""
    try:
        response = urlopen(download_url)
        file = open("pdf/"+filename+".pdf", 'wb')
        file.write(response.read())
        file.close()
    except:
        print(f"failed to download {filename}")

In [47]:
tools = [search_tool, scrape_pdf, download_pdf]

In [48]:
# RAG Model

In [49]:
class Model:
    # Define the model
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.classify)
        graph.add_node("action", self.take_action)
        #graph.add_node("output-parser", self.classify)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Check if llm requires action 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    # Run a tool ordered by the model
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    # Perform inference on the gathered context
    def classify(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # def parse_output_to_string(self, state: AgentState):
    #     message = state['messages'][-1]
    #     json_output = JsonOutputParser(pydantic_object=Restaurants).invoke(message)
    #     return {'json_output': json_output}

In [50]:
prompt = """You are tasked with finding and downloading a PDF of a restaurant's menu. 
Use the search tool to find the url of the menu, if you cannot find the pdf url directly use the scrape_pdf tool to scrape webpages for the url.
DO NOT attempt to download blank menus. Save the file using the PDF download tool, name the PDF after the restaurant.
Once you have downloaded the PDF there are no more tasks to be done. 
"""

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
)

model = Model(llm, tools, system=prompt)

In [57]:
messages = ["Mondo, Middletown Connecticut"]
result = model.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Mondo restaurant menu Middletown Connecticut PDF'}, 'id': 'call_VJKAxel82mdKQrjO4im0OPRW', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'scrape-pdf', 'args': {'url': 'https://www.mondomiddletown.com/menu'}, 'id': 'call_c1V7rO1Kh7KdR5WJRk1nvrYy', 'type': 'tool_call'}
Calling: {'name': 'scrape-pdf', 'args': {'url': 'https://www.sirved.com/restaurant/middletown-connecticut-usa/mondo/588117/menus'}, 'id': 'call_edYRRxgchv5I5lxByahQjDOh', 'type': 'tool_call'}


AttributeError: 'NoneType' object has no attribute 'endswith'

In [54]:
result

{'messages': ['Mondo, Middletown Connecticut',
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nsE08MvqhVNQMFTrCC1qwekB', 'function': {'arguments': '{"query":"Mondo restaurant menu PDF Middletown Connecticut"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 217, 'total_tokens': 243}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-46a49c53-f18f-4a92-9695-5acbb204d85f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Mondo restaurant menu PDF Middletown Connecticut'}, 'id': 'call_nsE08MvqhVNQMFTrCC1qwekB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 26, 'total_tokens': 243}),
  ToolMessage(content='[{\'url\': \'https://www.mondomiddletown.com/menu\', \'content\': \'Open Menu Close Menu. Home Menu Folder: Order. Back. Takeout or Delive

In [17]:
import pygraphviz
Image(model.graph.get_graph().draw_png())

ModuleNotFoundError: No module named 'pygraphviz'

In [ ]:
# Datastore

In [ ]:
# Tokenize the pdf 
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")




In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

pc.create_index(
    name="quickstart",
    dimension=5120, # Lamma 2 embedding dim
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)